<h3> Leetcode Hard SQL Problem - 6 | Second Most Recent Activity | SQL Window Analytical Functions </h3>

In [0]:
%sql
create table UserActivity
(
username      varchar(20) ,
activity      varchar(20),
startDate     Date   ,
endDate      Date
);

insert into UserActivity values 
('Alice','Travel','2020-02-12','2020-02-20')
,('Alice','Dancing','2020-02-21','2020-02-23')
,('Alice','Travel','2020-02-24','2020-02-28')
,('Bob','Travel','2020-02-11','2020-02-18');

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from UserActivity

username,activity,startDate,endDate
Alice,Travel,2020-02-12,2020-02-20
Alice,Dancing,2020-02-21,2020-02-23
Alice,Travel,2020-02-24,2020-02-28
Bob,Travel,2020-02-11,2020-02-18


<h3> Solution in sql(hive) </h3>

In [0]:
%sql 
select username,activity,startDate,endDate from(
select *,count((1))over(partition by username )cnt,row_number()over(partition by username order by startDate)rnk from UserActivity)a1
where cnt=1 or rnk=2

username,activity,startDate,endDate
Alice,Dancing,2020-02-21,2020-02-23
Bob,Travel,2020-02-11,2020-02-18


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import col,count,row_number
from pyspark.sql.window import Window

In [0]:
df_u=spark.sql("select * from UserActivity")
df_u.show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice|  Travel|2020-02-12|2020-02-20|
|   Alice| Dancing|2020-02-21|2020-02-23|
|   Alice|  Travel|2020-02-24|2020-02-28|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+



In [0]:
df_u=df_u.withColumn("rn",row_number().over(Window.partitionBy("username").orderBy("startDate"))).\
    withColumn("cnt",count("*").over(Window.partitionBy("username")))
df_u.filter((col("rn")==2)|(col("cnt")==1)).select("username","activity","startDate","endDate").show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice| Dancing|2020-02-21|2020-02-23|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+

